# Spotify Music Recommendation System with Mood Detection Using The Million Song Dataset

Author: James Meredith

Instructor: David Elliot

Institution: Flatiron School

Active Project Dates: Jun 267th, 2023 - July 14th, 2023

## Overview

Recommendation systems use machine learning to solve the long-tail problem with digital content. There is a lot of content out on the internet so it would take a person a long time to sort through it to find something they like. Recommendation systems help wth that buy suggesting content to people. [Netflix estimates](https://towardsdatascience.com/deep-dive-into-netflixs-recommender-system-341806ae3b48) it gets around 80% of total watch time thanks to its recommendation system.

The data is fairly clean already so most of the preperation involved manipulating the datasets into formats that can be used by recommendation systems, some recommendation systems use the format provided but ALS by hand and cosign similarity required some modifications.

Our modeling approach was:

- Start with a content-based system using cosign similarity, a commonly used distance measurement for recommendation systems
- Move to collaborative filtering using surprise, the popular library for implementing predictive recommendation systems
- Introduce ALS by hand to cement our understanding of the approach
- Implement ALS in Spark using pyspark separately, spark is commonly used to implement ALS at scale

We evaluated these models using RMSE as the measurement. Netflix's famous competition that offered $1M to a winner that could improve their content recommendation algorythm by 10% RMSE which was finally won by BellKor’s Pragmatic Chaos in 2009 with an RMSE of 0.8567. We decided that was our goal. All models were trained on train data, cross validated, and then evaluated on test data.

Note: ratings were on a scale of 1-5 so a RMSE of 1 means the average prediction was off by a whole score

Our best performing model was SVD optimized using gridsearch using surprise. Our recommendation however is to use ALS via spark to start however because of the efficiency of ALS and Spark in handling large datasets.

## Business Problem

## The Data

The dataset used for this project was the Million Song Database. It contains metadata for one million songs. The metadata includes information such as artist, title, year, and genre. The dataset also contains audio features for each song. The audio features are extracted from the audio files using the Echo Nest analyzer. The audio features include information such as tempo, loudness, and key. The dataset also contains lyrics for each song. The lyrics are extracted from the audio files using the Musixmatch analyzer. The lyrics are stored as a bag of words. The dataset is stored in HDF5 format. The dataset is 280 GB uncompressed. The dataset is available for download at http://millionsongdataset.com/. The dataset is also available on AWS S3 at s3://millionsongdataset/. 

The Million Song Dataset is a freely-available collection of audio features and metadata for a million contemporary popular music tracks. The Million Song Dataset started as a collaborative project between The Echo Nest and LabROSA. It was supported in part by the NSF. 

Its purposes are:

- To encourage research on algorithms that scale to commercial sizes
- To provide a reference dataset for evaluating research
- As a shortcut alternative to creating a large dataset with APIs (e.g. The Echo Nest's)
- To help new researchers get started in the MIR field

The core of the dataset is the feature analysis and metadata for one million songs, provided by The Echo Nest. 

Thierry Bertin-Mahieux, Daniel P.W. Ellis, Brian Whitman, and Paul Lamere. 
The Million Song Dataset. In Proceedings of the 12th International Society
for Music Information Retrieval Conference (ISMIR 2011), 2011.

### Exploring The Dataset

In [20]:
# Imports the required libraries for the project
import os
import sys
import time
import glob
import datetime
import sqlite3
import numpy as np
import pandas as pd
import tables as tb

# Standard imports
import pandas as pd
import numpy as np
from random import gauss, uniform as uni, seed
import math as math
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Cosign Similarity
from sklearn.metrics.pairwise import cosine_similarity

# Surprise imports for collaborative filtering
from surprise import Dataset, Reader
from surprise.prediction_algorithms import *
from surprise.model_selection import cross_validate, train_test_split, GridSearchCV
from surprise.accuracy import rmse, mae

# ALS
from sklearn.linear_model import LinearRegression

The structure of the dataset is quite complex, with many nested folders with files within them. The first step is to explore the structure of the dataset. Here we use the os module to explore the structure of the dataset.

In [21]:
# Establishes path to the Million Song Dataset subset
msd_subset_path='./MillionSongSubset'
msd_subset_data_path=os.path.join(msd_subset_path,'data')
msd_subset_addf_path=os.path.join(msd_subset_path,'AdditionalFiles')

# Establishes path to the Million Song Dataset code
msd_code_path='./MSongsDB'
assert os.path .isdir(msd_code_path),'wrong path' # sanity check
# we add some paths to python so we can import MSD code
# Ubuntu: you can change the environment variable PYTHONPATH
# in your .bashrc file so you do not have to type these lines
sys.path.append( os.path.join(msd_code_path,'PythonSrc') )

# Creates a function to display the folder structure of the dataset
def display_folder_structure(path, indent='', max_depth=float('inf'), depth=0):
    if depth > max_depth:
        return

    files = os.listdir(path)
    for file in files:
        current_path = os.path.join(path, file)
        if os.path.isdir(current_path):
            print(f"{indent}|- {file}/")
            display_folder_structure(current_path, indent + '  ', max_depth, depth + 1)
        else:
            print(f"{indent}|- {file}")


# Uses the function to display the folder structure of the dataset to the specified depth
max_display_depth = 0  # Specify the maximum depth to display
display_folder_structure(msd_subset_path, max_depth=max_display_depth)

|- A/
|- B/


So the structure of the dataset is as follows:

- The dataset is stored in the folder `MillionSongSubset`
- The individual dataset files are stored in HDF5 format
- The dataset is stored in a nested folder structure
- The dataset contains 10,000 folders, each containing 100 folders, each containing 100 files

The next step is to explore the contents of the dataset. Here we use the h5py module to explore the contents of the dataset.


In [22]:
# Let's open one file and explore it's structure and contents
filename = '.\MillionSongSubset\A\A\A\TRAAAAW128F429D538.h5'
h5file = tb.open_file(filename, mode='r')
h5file 

File(filename=.\MillionSongSubset\A\A\A\TRAAAAW128F429D538.h5, title='H5 Song File', mode='r', root_uep='/', filters=Filters(complevel=1, complib='zlib', shuffle=True, bitshuffle=False, fletcher32=False, least_significant_digit=None))
/ (RootGroup) 'H5 Song File'
/analysis (Group) 'Echo Nest analysis of the song'
/analysis/bars_confidence (EArray(83,)shuffle, zlib(1)) 'array of confidence of bars'
  atom := Float64Atom(shape=(), dflt=0.0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'little'
  chunkshape := (1024,)
/analysis/bars_start (EArray(83,)shuffle, zlib(1)) 'array of start times of bars'
  atom := Float64Atom(shape=(), dflt=0.0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'little'
  chunkshape := (1024,)
/analysis/beats_confidence (EArray(344,)shuffle, zlib(1)) 'array of confidence of sections'
  atom := Float64Atom(shape=(), dflt=0.0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'little'
  chunkshape := (1024,)
/analysis/beats_start (EArray(344,)shuffle, zlib(1

In [30]:
h5file.root

/ (RootGroup) 'H5 Song File'
  children := ['analysis' (Group), 'metadata' (Group), 'musicbrainz' (Group)]

In [32]:
h5file.root.analysis


/analysis (Group) 'Echo Nest analysis of the song'
  children := ['bars_confidence' (EArray), 'bars_start' (EArray), 'beats_confidence' (EArray), 'beats_start' (EArray), 'sections_confidence' (EArray), 'sections_start' (EArray), 'segments_confidence' (EArray), 'segments_loudness_max' (EArray), 'segments_loudness_max_time' (EArray), 'segments_loudness_start' (EArray), 'segments_pitches' (EArray), 'segments_start' (EArray), 'segments_timbre' (EArray), 'songs' (Table), 'tatums_confidence' (EArray), 'tatums_start' (EArray)]

In [33]:
h5file.root.metadata

/metadata (Group) 'metadata about the song'
  children := ['artist_terms' (EArray), 'artist_terms_freq' (EArray), 'artist_terms_weight' (EArray), 'similar_artists' (EArray), 'songs' (Table)]

In [34]:
h5file.root.musicbrainz

/musicbrainz (Group) 'data about the song coming from MusicBrainz'
  children := ['artist_mbtags' (EArray), 'artist_mbtags_count' (EArray), 'songs' (Table)]

So the structure of each .h5 file is as follows:
- The file contains 3 groups: analysis, metadata, and musicbrainz
- The analysis group contains:
    bars_confidence, bars_start, beats_confidence, beats_start, sections_confidence, sections_start, segments_confidence, segments_loudness_max, segments_loudness_max_time, segments_loudness_start, segments_pitches, segments_start, segments_timbre, songs, tatums_confidence, and tatums_start
- The metadata group contains:
    artist_terms, artist_terms_freq, artist_terms_weight, similar_artists, songs, and tags
- The musicbrainz group contains:
    artist_mbtags, artist_mbtags_count, songs, and tags


## Data Preperation & Cleaning

Because the structure of the dataset is catered towards big data applications, it is not ideal for machine learning applications. The first step is to convert the dataset into a more machine learning friendly format. Here we use the pandas module to convert the dataset into a pandas dataframe.

In [27]:
df = pd.DataFrame(h5file)
df

,0,1,2,3,4,5,6,7,8,9,...,961,962,963,964,965,966,967,968,969,970
0,/analysis (Group) 'Echo Nest analysis of the s...,/metadata (Group) 'metadata about the song',/musicbrainz (Group) 'data about the song comi...,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,"/analysis/bars_confidence (EArray(83,)shuffle,...","/analysis/bars_start (EArray(83,)shuffle, zlib...","/analysis/beats_confidence (EArray(344,)shuffl...","/analysis/beats_start (EArray(344,)shuffle, zl...","/analysis/sections_confidence (EArray(10,)shuf...","/analysis/sections_start (EArray(10,)shuffle, ...","/analysis/segments_confidence (EArray(971,)shu...","/analysis/segments_loudness_max (EArray(971,)s...",/analysis/segments_loudness_max_time (EArray(9...,"/analysis/segments_loudness_start (EArray(971,...",...,None,None,None,None,None,None,None,None,None,None
2,"/metadata/artist_terms (EArray(37,)shuffle, zl...","/metadata/artist_terms_freq (EArray(37,)shuffl...","/metadata/artist_terms_weight (EArray(37,)shuf...","/metadata/similar_artists (EArray(100,)shuffle...","[/metadata/songs.row (Row), pointing to row #0]",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,"/musicbrainz/artist_mbtags (EArray(0,)shuffle,...","/musicbrainz/artist_mbtags_count (EArray(0,)sh...","[/musicbrainz/songs.row (Row), pointing to row...",None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,0.643,0.746,0.722,0.095,0.091,0.362,0.465,0.204,0.129,0.618,...,None,None,None,None,None,None,None,None,None,None
5,0.58521,2.94247,5.14371,7.74554,10.36149,12.98399,15.59835,18.21002,20.81724,23.41491,...,None,None,None,None,None,None,None,None,None,None
6,0.834,0.851,0.65,0.635,0.532,0.753,0.622,0.657,0.704,0.745,...,None,None,None,None,None,None,None,None,None,None
7,0.58521,1.19196,1.78893,2.37813,2.94247,3.50622,4.05077,4.56902,5.14371,5.76504,...,None,None,None,None,None,None,None,None,None,None
8,1.0,1.0,0.218,0.133,0.384,0.326,0.373,0.129,0.588,0.62,...,None,None,None,None,None,None,None,None,None,None
9,0.0,7.74554,36.44331,43.61667,75.17954,90.1827,135.77195,164.23964,189.03133,198.20273,...,None,None,None,None,None,None,None,None,None,None


In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Columns: 971 entries, 0 to 970
dtypes: object(971)
memory usage: 212.5+ KB


Attempting to import an individual data file wholesale into a pandas dataframe results in a dataframe with 971 columns and 27 rows - for just one song! Multiply that by 10,000 for the number of songs in the dataset and you have a dataframe with 9,710,000 columns and 270,000 rows - for just the Million Song Subset! This is not ideal for loading into memory on a local machine. Instead we will only load the columns we need into the dataframe moving forward. We'll start by defining a function that can iterate through the dataset and load the columns we need into a dataframe, and then prepare the dataframe for machine learning applications.

In [25]:
# we define this very useful function to iterate through all the files in the dataset
def apply_to_all_files(basedir,func=lambda x: x,ext='.h5'):
    """
    From a base directory, go through all subdirectories,
    find all files with the given extension, apply the
    given function 'func' to all of them.
    If no 'func' is passed, we do nothing except counting.
    INPUT
       basedir  - base directory of the dataset
       func     - function to apply to all filenames
       ext      - extension, .h5 by default
    RETURN
       number of files
    """
    cnt = 0
    # iterate over all files in all subdirectories
    for root, dirs, files in os.walk(basedir):
        files = glob.glob(os.path.join(root,'*'+ext))
        # count files
        cnt += len(files)
        # apply function to all files
        for f in files :
            func(f)       
    return cnt

In [26]:
# we can now easily count the number of files in the dataset
print('number of song files:',apply_to_all_files(msd_subset_path))

number of song files: 10000


## Data Exploration

## Modeling

### Content-Based Model using Cosine Similarity

### Collaborative Filtering

### Aleternating Least-Squares (ALS)

## Conclusion:

- Content based systems are simple to implement but hard to measure error (no RMSE)
- Collaborative filtering has issues with cold start (movies with no ratings)
- Collaborative filtering using surprise is incredibly effective and produces great models
- ALS is incredibly efficient an simple

There are many options to create recommendation systems. 
In this notebook we explored content based systems using cosign similarity, 
various collaborative filtering models via surprise,
and dove deep into the concepts behind ALS to explain how it works.

We were able to beat BellKor’s Pragmatic Chaos's RMSE of 0.8567 with an RMSE of 0.8559 but this was likely because of our smaller dataset.

The best approach to recommendation systems is likely a hybrid approach which we will not explore in this notebook.